### Configuration

```bash
brew install apache-spark
pip install pyspark
```

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, max, avg, count
from pyspark.sql import SparkSession

### First script to run in order to make the queries run

In [16]:
# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("SpotifyGenre") \
      .getOrCreate()
# master contains the URL of your remote spark instance or 'local'

### Create the Schema

In [17]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType

# Define the schema for your CSV data
schemaSpotifyGenre = StructType([
    StructField("id", IntegerType(),True),
    StructField("track_id", StringType(), True),
    StructField("artists", StringType(), True),
    StructField("album_name", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("popularity", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("explicit", BooleanType(), True),
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("time_signature", IntegerType(), True),
    StructField("track_genre", StringType(), True)
])

### Parse the Dataset

In [ ]:
df = spark.read.csv('../data/spotify_tracks_genre.csv', header=True, schema=schemaSpotifyGenre)
df.printSchema()
# df.show()  # To display the first few rows of the dataframe

### Python Query 1: Danceability Statistic for a Specific Author and Genre

In [ ]:
# Filter for 'Bad Bunny' in the artist column and 'acoustic' in the genre column
filtered_df = df.filter(
    col("artists").like("Bad Bunny")
)

# Select the 'danceability' column and calculate the average, minimum, and maximum
aggregated_df = filtered_df.agg(
    avg("danceability").alias("avg_danceability"),
    min("danceability").alias("min_danceability"),
    max("danceability").alias("max_danceability")
)

# Show the results
aggregated_df.show()

### Python Query 2: Explicit Authors
This query aims to find all the authors that have published at least an explicit song.\
They are then sorted by the number of explicit songs published.

In [ ]:
result = (
    df.filter(col('explicit') == True)  # Filter for explicit songs
    .groupBy('artists')  # Group by artists
    .agg(count('track_id').alias('explicit_count'))  # Count the number of explicit songs for each artist
    .orderBy(col('explicit_count').desc(), 'artists')  # Sort by explicit count in descending order, then alphabetically
)

result.show()

### Python Query 3: Long and Soft Tracks

In [ ]:
# Filter the DataFrame for tracks longer than 240000 milliseconds and with loudness less than -10.0 dB
filtered_df = df.filter((col("duration_ms") > 240000) & (col("loudness") < -10.0))

# Select the desired columns
selected_df = filtered_df.select('track_id', 'artists', 'track_name', 'duration_ms', 'loudness')

# Show the results
selected_df.show()

### Python Query 4: Best Danceable Dance Songs

This query aims to find all the danceable songs that are not explicit.\
It only shows the songs that have a danceability that's more than 0.8.


In [ ]:
genre = 'dance'  # Replace 'dance' with your desired genre

result = (
    df.filter(
        (col('explicit') == False) & 
        (col('danceability') > 0.8) & 
        (col('track_genre').contains(genre))
    )  # Filter for non-explicit dance songs with danceability > 0.8 and specific genre
    .select('track_name', 'danceability')  # Select only 'track_name' and 'danceability' columns
    .orderBy('danceability', ascending=False)  # Sort by danceability in descending order
)

result.show(truncate=False)

### Python Query 5: Understated Upbeat Tracks

In [ ]:
# Filter the DataFrame for tracks with popularity less than 40 and valence greater than 0.7
filtered_df = df.filter((col("popularity") < 40) & (col("popularity") > 10) & (col("valence") > 0.7))

# Select the desired columns
selected_df = filtered_df.select('track_name', 'artists', 'popularity', 'valence')

# Show the results
selected_df.show()

### SQL Queries

### SQL Query 1: Top 5 Longest Albums in Dataset
This query aims to find the top 5 longest albums in the Dataset.\
At first the songs are grouped by their album name, then the duration time of each song is summed up.\
Then the top 5 are showed and sorted in a Descending order.

In [ ]:
# Register the DataFrame as a temporary SQL table
df.createOrReplaceTempView("spotify_genre_table")

# SQL query to identify albums with the longest total duration
sql_query = """
    SELECT album_name, SUM(duration_ms) AS total_duration
    FROM spotify_genre_table
    GROUP BY album_name
    ORDER BY total_duration DESC
    LIMIT 5
"""

# Execute the SQL query
result = spark.sql(sql_query)

result.show(truncate=False)

### SQL Query 2: Highly Instrumental Tracks

In [ ]:
# Register the DataFrame as a temporary table
df.createOrReplaceTempView("tracks")

sql_query = """
    SELECT track_id, track_name, instrumentalness, speechiness
    FROM tracks
    WHERE instrumentalness > 0.5 AND speechiness < 0.3
"""

# Execute the SQL query
result = spark.sql(sql_query)

result.show()

### SQL Query 3: Top 5 Popular Genres

This query aims to find the top 5 genres by popularity.\
At first it groups all the songs by genre and computes the average popularity.
Then genres are sorted by popularity in descending order.

In [ ]:
# Put query here FRUAH
df.createOrReplaceTempView("spotify_genre_table")

# SQL query to identify albums with the longest total duration
sql_query = """
    SELECT track_genre, AVG(popularity) AS avg_popularity
    FROM spotify_genre_table
    GROUP BY track_genre
    ORDER BY avg_popularity DESC
    LIMIT 5
"""
# Execute the SQL query
result = spark.sql(sql_query)

result.show(truncate=False)